<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

# MLP Modelle trainieren

In diesem Aufgabenblatt geht es darum, MLP-Modelle mit *Scikit-learn* und *Keras* aufzustellen und zu trainieren.
Wir verwenden dazu einen synthetischen, nicht-linear-separierbaren Datensatz mit 3 Klassen von Datenpunkten.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def make_spirals(N=100, classes=2, random_state=0):
    np.random.seed(random_state)
    X = np.zeros((N*classes,2))
    num_train_examples = X.shape[0]
    y = np.zeros(N*classes, dtype='uint8')
    for j in range(classes):
        ix = range(N*j,N*(j+1))
        r = np.linspace(0.0,1,N) # radius
        k = classes+1
        t = np.linspace(j*k,(j+1)*k,N) + np.random.randn(N)*0.2 # theta
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    return X, y

N=60
classes=3
random_state=0

X, y = make_spirals(N, classes, random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


fig = plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim([-1,1])
plt.ylim([-1,1])

Wie man in folgendem Beispiel sieht, schneidet ein Lineares Modell bei diesen Daten sehr schlecht ab:

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, multi_class='ovr').fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
def plot_decregions_2d(X,y,clf):
    '''
    Plotte Entscheidungsgrenzen in einem
    Datensatz mit zwei Attributen
    
    Parameter:
    X: 2d numpy array der Groesse (m,n)
    clf: Klassifizierer mit einer Methoder "predict"
    '''
    cmap = plt.get_cmap('Set1', 3)
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    print(Z.shape)
    if Z.ndim>1:
        Z = np.argmax(Z, axis=1)
        print(Z.shape)
    Z = Z.reshape(xx.shape)
    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, ec='black', cmap=cmap)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

In [ ]:
plot_decregions_2d(X_train, y_train, clf)

**Aufgabe:** Trainieren Sie einen MLPClassifier (aus SKlearn) für den Datensatz. Das MLP soll einen *hidden layer* mit 32 Neuronen besitzen. Welche *Classification Accuracy* Erreicht Ihr Modell für den Trainigsdatensatz? Plotten Sie die Entscheidungsgrenzen Ihres Modells.

In [ ]:
from sklearn.neural_network import MLPClassifier
# YOUR CODE HERE
raise NotImplementedError()

**Aufgabe:** Bestimmen Sie aus einer Auswahl vorgegebener Hyperparameter die optimale Kombination für einen *MLPClassifier* mithilfe der *GridSearchCV*-Methode. Folgende Parameter sollen zur Auswahl stehen:
* `hidden_layer_sizes`: Dieser Parameter gibt die Größe der versteckten Schichten des neuronalen Netzwerks an. Folgende Optionen sollen betrachtet werden:
 - `(32,)`: Eine versteckte Schicht mit 32 Neuronen.
 - `(24, 8)`: Zwei versteckte Schichten mit 24 und 8 Neuronen.
 - `(8, 24)`: Zwei versteckte Schichten mit 8 und 24 Neuronen.
 - `(16, 8, 8)`: Drei versteckte Schichten mit 16, 8 und 8 Neuronen.
 - `(8, 16, 8)`: Drei versteckte Schichten mit 8, 16 und 8 Neuronen.
 - `(16, 8, 4, 4)`: Vier versteckte Schichten mit 16, 8, 4 und 4 Neuronen.
 

* `activation`: Dieser Parameter definiert die Aktivierungsfunktion, die in den Neuronen verwendet wird. Folgende Optionen sollen betrachtet werden:
 - `tanh`: Tangens hyperbolicus
 - `relu`: ReLU (Rectified Linear Unit)
 - `logistic`: Logistische Funktion
 
 
* `solver`: Dieser Parameter legt den Optimierungsalgorithmus fest, der für das Lernen der Gewichte verwendet wird. Folgende Optionen sollen betrachtet werden:
 - `sgd`: Stochastic Gradient Descent (SGD) Optimierungsalgorithmus.
 - `adam`: Adam Optimierungsalgorithmus.
 
 
* `alpha`: Dieser Parameter bestimmt den Regularisierungsterm, der zur Vermeidung von Überanpassung verwendet wird. Folgende Optionen sollen betrachtet werden:
 - `0.0001`
 - `0.01`
 - `0.05`
 
 
* `learning_rate`: Dieser Parameter legt die Lernrate fest, die den Gradientenabstieg beeinflusst. Folgende Optionen sollen betrachtet werden:
 - `constant`: Konstante Lernrate.
 - `adaptive`: Adaptive Lernrate.


Die möglichen Parameter sollen in einem Dictionary `parameter_space` angegben werden. Der Parameter-Name ist dabei jeweils der Key. Der Value ist eine Liste der möglichen Werte.

Führen Sie die Suche zuerst für eine maximale Anzahl von 20 Iterationen durch. Ändern sich die optimalen Parameter, wenn Sie 100 und 500 Iterationen lang trainieren?

In [ ]:
import sys, os, warnings
# Das hier ist gefährlich, weil man so Fehler übersehen kann!!!
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"


from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


parameter_space = None
clf = None
epochs = 20


# YOUR CODE HERE
raise NotImplementedError()


print(f'Die besten Parameter nach {epochs} Epochen:\n', clf.best_params_)

acc = clf.best_estimator_.score(X_test, y_test)
print(f'Accuracy nach {epochs} Epochen: {acc}')
plot_decregions_2d(X_train, y_train, clf.best_estimator_);

